In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kilianovski/spoonvsfork")

print("Path to dataset files:", path)

100%|██████████| 9.07M/9.07M [00:00<00:00, 78.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kilianovski/spoonvsfork/versions/1


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import random
import pathlib
import tensorflow as tf
import IPython.display as display

# Cek apakah eager execution aktif
print("Eager execution aktif:", tf.executing_eagerly())

# Cek apakah GPU tersedia
print("GPU tersedia:", len(tf.config.list_physical_devices('GPU')) > 0)


Eager execution aktif: True
GPU tersedia: False


In [3]:
import tensorflow as tf
import numpy as np
import random

# Set seed agar hasil acak bisa direproduksi
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


In [4]:
import os
from os.path import join

# Langkah 1: Dapatkan path dari kagglehub
path = kagglehub.dataset_download("kilianovski/spoonvsfork")
print("Path dari kagglehub:", path)

# Langkah 2: Telusuri isi folder
print("Isi dari path:", os.listdir(path))

# Langkah 3: Masuk ke subfolder 'spoon-vs-fork'
basedir = join(path, 'spoon-vs-fork')
print("Isi basedir:", os.listdir(basedir))

# Langkah 4: Akses folder spoon dan fork
fork_dir = join(basedir, 'fork')
spoon_dir = join(basedir, 'spoon')

# Langkah 5: Baca semua gambar
spoon_paths = [join(spoon_dir, f) for f in os.listdir(spoon_dir) if f.endswith(('.jpg', '.png'))]
fork_paths = [join(fork_dir, f) for f in os.listdir(fork_dir) if f.endswith(('.jpg', '.png'))]
img_paths = spoon_paths + fork_paths

print("Jumlah gambar sendok:", len(spoon_paths))
print("Jumlah gambar garpu :", len(fork_paths))
print("Total gambar        :", len(img_paths))


Path dari kagglehub: /kaggle/input/spoonvsfork
Isi dari path: ['spoon-vs-fork']
Isi basedir: ['spoon', 'fork', 'spoon-vs-fork']
Jumlah gambar sendok: 144
Jumlah gambar garpu : 186
Total gambar        : 330


In [5]:
import os
import pandas as pd
from os.path import join

def load_data(basedir):
    folders = [f for f in os.listdir(basedir) if os.path.isdir(join(basedir, f))]
    print("Folder label ditemukan:", folders)

    result = pd.DataFrame(columns=['filename', 'class'])

    for folder in folders:
        folder_path = join(basedir, folder)
        files = [join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('.jpg', '.png'))]
        df = pd.DataFrame({'filename': files, 'class': folder})
        result = pd.concat([result, df], ignore_index=True)

    return result


In [6]:
basedir = join(path, 'spoon-vs-fork')  # dari sebelumnya

image_df = load_data(basedir)
print(image_df.head())
print("Total gambar:", len(image_df))
print("Kelas unik:", image_df['class'].unique())


Folder label ditemukan: ['spoon', 'fork', 'spoon-vs-fork']
                                            filename  class
0  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
1  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
2  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
3  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
4  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
Total gambar: 330
Kelas unik: ['spoon' 'fork']


In [7]:
def validate_data(image_df):
    allowed_extensions = ['jpg', 'jpeg', 'png', 'gif']

    def is_valid_image(filename):
        extension = os.path.splitext(filename)[1][1:].lower()
        return extension in allowed_extensions

    invalid_files = image_df[~image_df['filename'].apply(is_valid_image)]

    # Log file yang dihapus
    for img in invalid_files['filename']:
        ext = os.path.splitext(img)[1][1:].lower()
        print(f"Removed file with extension '{ext}' — {img}")

    # Hapus baris tidak valid
    return image_df[image_df['filename'].apply(is_valid_image)].reset_index(drop=True)


In [8]:
image_df = validate_data(image_df)
print("Jumlah gambar valid:", len(image_df))
print(image_df.head())


Jumlah gambar valid: 330
                                            filename  class
0  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
1  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
2  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
3  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon
4  /kaggle/input/spoonvsfork/spoon-vs-fork/spoon/...  spoon


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    image_df.filename,
    image_df['class'],
    test_size=0.2,
    random_state=42,
    stratify=image_df['class']  # menjaga proporsi spoon vs fork
)


In [11]:
train_df = pd.DataFrame({'filename': X_train, 'class': y_train})
test_df = pd.DataFrame({'filename': X_test, 'class': y_test})

print("Jumlah data latih:", len(train_df))
print("Jumlah data uji  :", len(test_df))
print("Distribusi kelas (train):")
print(train_df['class'].value_counts())


Jumlah data latih: 264
Jumlah data uji  : 66
Distribusi kelas (train):
class
fork     149
spoon    115
Name: count, dtype: int64


In [12]:
from tensorflow.keras.applications import ResNet50

resnet = ResNet50(include_top=False, pooling='avg', weights='imagenet')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [13]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

# Inisialisasi ResNet50 tanpa top layer, dengan bobot ImageNet
resnet = ResNet50(include_top=False, pooling='avg', weights='imagenet')

# Bangun model klasifikasi 2 kelas
resnet_model = Sequential([
    resnet,
    Dense(2, activation='softmax')  # output 2 kelas: spoon dan fork
])

# Bekukan layer ResNet (tidak dilatih ulang)
resnet.trainable = False


In [14]:
resnet_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,591,810 (90.00 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

batch_size = 16

# Generator dengan augmentasi untuk training
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    rotation_range=10
)

# Generator tanpa augmentasi untuk validasi
valid_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# DataFrame training
train_df = pd.DataFrame({'filename': X_train, 'class': y_train})
test_df = pd.DataFrame({'filename': X_test, 'class': y_test})

# Flow training
train_flow = train_gen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    directory=None  # karena path-nya sudah full
)

# Flow validasi
valid_flow = valid_gen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=False,
    directory=None
)


Found 264 validated image filenames belonging to 2 classes.
Found 66 validated image filenames belonging to 2 classes.


In [16]:
from tensorflow.keras.optimizers import Adam

resnet_model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',  # karena class_mode='categorical'
    metrics=['accuracy']
)


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam

# Generator
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    rotation_range=10
)

valid_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow from dataframe
train_flow = train_gen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=16,
    shuffle=True
)

valid_flow = valid_gen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=16,
    shuffle=False
)

# Compile model
resnet_model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fit model (pakai fit, BUKAN fit_generator)
resnet_model.fit(
    train_flow,
    validation_data=valid_flow,
    epochs=8,
    steps_per_epoch=len(train_flow),
    validation_steps=len(valid_flow)
)


Found 264 validated image filenames belonging to 2 classes.
Found 66 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.6225 - loss: 0.6648 - val_accuracy: 0.8788 - val_loss: 0.2768
Epoch 2/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9601 - loss: 0.1567 - val_accuracy: 0.9394 - val_loss: 0.2092
Epoch 3/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9729 - loss: 0.1119 - val_accuracy: 0.9242 - val_loss: 0.2165
Epoch 4/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.9755 - loss: 0.0881 - val_accuracy: 0.9242 - val_loss: 0.2319
Epoch 5/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.9913 - loss: 0.0688 - val_accuracy: 0.9394 - val_loss: 0.2066
Epoch 6/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.9939 - loss: 0.0662 - val_accuracy: 0.9242 - val_loss: 0.2048
Epoch 7/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.9958 - loss: 0.0549 - val_accuracy: 0.9545 - val_loss: 0.1917
Epoch 8/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.9951 - loss: 0.0492 - val_accuracy: 0.9091 - val_loss: 0.2246
